<a href="https://colab.research.google.com/github/yoonsongg/CS2/blob/main/Deep_Learning_from_scratch3_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#step11 가변길이인수(순전파편) ~ step 13

In [ ]:
class Variable:
    def __init__(self, data):
        #only ndarray 만 가능하도록
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('f{}은 지원하지 않습니다.', type(data))

        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        #매번 y.grad = np.array(1.0) 쓰기 구찮(so 아래 두줄)
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
 
            if x.creator is not None:
                funcs.append(x.creator)


In [ ]:
class Function:
    def __call__(self, inputs):
        ## 리스트로 변경
        #x = input.data
        #y = self.forward(x)
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        #편의를 위한 함수 사용(as_array)
        #output = Variable(y)
        #output = Variable(as_array(y))

        #output.set_creator(self)
        #self.input = input
        #self.output = output

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs

    def forward(self, xs):
        raise NotImplementedError
    
    def backward(self, gys):
        raise NotImplementedError
    
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )


In [ ]:
#편의를 위한 함수
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

#매번 선언하기 귀찮. 그래서 다시 함수구현
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [ ]:
import numpy as np
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
print(ys[0])
print(ys[0].data)

5


#step12 가변길이 인수(개선편)

In [ ]:
def f(*x):
    print(x)

f(1,2,3, 4, 5)

(1, 2, 3, 4, 5)


In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [ ]:
import numpy as np 

class Variable:
    def __init__(self, data):
        #only ndarray 만 가능하도록
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        #매번 y.grad = np.array(1.0) 쓰기 구찮(so 아래 두줄)
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]

        while funcs:
            f = funcs.pop()
            #입출력 하나씩 -> 입출력 여러개
            # x, y = f.input, f.output
            # x.grad = f.backward(y.grad)
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)
 
            # if x.creator is not None:
            #     funcs.append(x.creator)

In [ ]:
#*inputs 해서 * 붙이면 가변길이 인수 받는 것 가능
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gys


def add(x0, x1):
    return Add()(x0, x1)

In [ ]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        #x = self.input.data
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [ ]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


#step14 같은 변수 반복 사용

In [ ]:
import numpy as np 

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]

        while funcs:
            f = funcs.pop()

            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                #전파되는 미분값의 합이 필요 한 것
                #x.grad = gx
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)
  #grad 초기화 해주는 메쏘드 추가 
    def cleargrad(self):
        self.grad = None




In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


In [ ]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad()
x = Variable(np.array(3.0))
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0


#step15~16 복잡한 계산 그래프 


In [ ]:
import numpy as np 

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
        #역전파 순서를 제대로 하기 위해 변수 추가 
        self.generation = 0
    
    def set_creator(self, func):
        self.creator = func
        #creator 만들어질 때마다 세대번호 추가 
        self.generation = func.generation + 1
    
    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

#        funcs = [self.creator]

        #추가시킨 부분
        funcs = []
        seen_set = set()
        #backward안의 중첩함수(함수를 generation 순대로 정렬하는 역할)
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            #Function class에서 output에 약함참조 사용. 그래서 변경
#            gys = [output.grad for output in f.outputs]
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
 #                   funcs.append(x.creator)
                    add_func(x.creator)


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        #generation큰걸 꺼냄
        self.generation = max([x.generation for x in inputs])


        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
    

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        #x = self.input.data
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [ ]:
#dummy function 준비
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function()
    #function에 g(제너레이션숫자) 배당
    f.generation = g
    funcs.append(f)


In [ ]:
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [ ]:
funcs.sort(key = lambda x: x.generation)
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [ ]:
f= funcs.pop()
f.generation

4

In [ ]:
def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [ ]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


#step17 메모리관계와 순환참조

In [ ]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
#        self.outputs = outputs 약한참조 사용하려고 아래로 바꿈 -> Variable gys 도 변경함
        self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
    

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

#step18 메모리 절약 모드 

In [ ]:

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def cleargrad(self):
        self.grad = None
# retain_grad = False 추가
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)



        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:

                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

In [ ]:
class Config:
    enable_backprop = True

In [ ]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
#아래 if문 추가. 그 아래 코드는 그 안으로 들어감. 
#그래서 config.enable_backprop = True 일 때만 역전파 코드 실행

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
    

In [ ]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start') #전처리
    try:
        yield
    finally:
        print('done') #후처리

with config_test():
    print('process..')

#with 문 시작 : config_test METHOD 안의 전처리 -> with문 안 -> config_test 안의 후처리

start
process..
done


In [ ]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [ ]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [ ]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)



#step19 변수 사용성 개선

In [ ]:
import numpy as np
class Variable:
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
#add name 
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def cleargrad(self):
        self.grad = None
# retain_grad = False 추가
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:

                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + '' * 9)
        return 'variable(' + p + ')'


In [ ]:
x = Variable(np.array([[1,2,3], [4,5,6]]))
print(x.shape)
print(x.size)
print(x.dtype)
print(len(x))

(2, 3)
6
int64
2


In [ ]:
x = Variable(np.array([1,2,3]))
print(x)
#<__main__.Variable object at 0x7f009be023d0>


variable([1 2 3])


In [ ]:
x = Variable(None)
print(x)
#<__main__.Variable object at 0x7f009be5c990>

variable(None)


In [ ]:
x = Variable(np.array([[1,2,3], [4,5,6]]))
print(x)
#<__main__.Variable object at 0x7f009be826d0>

variable([[1 2 3]
 [4 5 6]])


#step20 연산자 오버로드


In [ ]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

def mul(x0, x1):
    return Mul()(x0, x1)

In [ ]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b), c)

y.backward()

In [ ]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [ ]:

class Variable:
    __array_priority__ = 200
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')

        self.data = data
#add name 
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def cleargrad(self):
        self.grad = None
# retain_grad = False 추가
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:

                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + '' * 9)
        return 'variable(' + p + ')'
    
    def __mul__(self, other):
        return mul(self, other)

Variable.__mul__ = mul
Variable.__add__ = add

Variable.__radd__ = add
Variable.__rmul__ = mul

In [ ]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [ ]:
class Function:
    def __call__(self, *inputs):
        #as_variable 함수 사용해서 inputs를 Variable 인스턴스로 바꿔줌 
        inputs = [as_variable(x) for x in inputs]
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
    

In [ ]:
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

x = Variable(np.array(2.0))
y = -x 
print(y)

variable(-2.0)


In [ ]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

In [ ]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

Variable.__rsub__ = rsub

In [ ]:
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0
print(y1)
print(y2)

variable(0.0)
variable(1.0)


In [ ]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)


Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv


In [ ]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow


In [ ]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)

variable(8.0)


In [ ]:
print(pow(np.array(2.0), np.array(3.0) ))

variable(8.0)


In [ ]:
#위랑 차이
print(pow(2.0, 3.0))

TypeError: ignored

In [ ]:
x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

TypeError: ignored

#step24

In [ ]:
class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


In [ ]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

In [ ]:
def sphere(x, y):
    z = x ** 2 + y ** 2
    return z


def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z


def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0
